<a href="https://colab.research.google.com/github/knallathambi-dev/llm-document-qna/blob/main/notebooks/Mistral7llamaindexDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pypdf
!pip install -q python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 3.4 MB/s eta 0:00:00


In [2]:
!pip install -q transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 100.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.6 MB/s eta 0:00:00


In [3]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 37.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.13-cp310-cp310-manylinux_2_35_x86_64.whl size=4094639 sha256=923fc22a71406d4d875d6e098358fea472598a7675caa932f69f7f278f26ded0
  Stored in directory: /tmp/pip-ephem-wheel-cache-_jh2j5f4/wheels/d3/c9/89/ec02bbfa2283812eb24639ba52e929b9d773f86e4419a7da58
Successfully built llama-cpp-python


In [6]:
!pip install -q llama-index


In [5]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext


In [7]:
documents = SimpleDirectoryReader("/content/sample_pdf/").load_data()

In [8]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)


total size (MB): 4368.44


4167it [00:22, 183.92it/s]                          
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [9]:
!pip -q install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.4 MB/s eta 0:00:00


In [10]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)


In [11]:
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [13]:
query_engine = index.as_query_engine()
response = query_engine.query("What business Amazon is in ?")

In [14]:
print(response)

 Based on the given context information, it appears that Amazon is primarily in the e-commerce business, offering a wide range of products and services such as books, electronics, clothing, and more. They also offer other services such as compute, storage, and database offerings, fulfillment, advertising, publishing, and digital content subscriptions. Additionally, they have registered or applied for trademarks, service marks, copyrights, patents, domain names, trade dress, trade secrets, proprietary technologies, and confidentiality and/or license agreements with employees, customers, partners, and others to protect their proprietary rights.


In [ ]:
while True:
  query=input()
  response = query_engine.query(query)
  print(response)

Tell me the trading symbol for amazon


Llama.generate: prefix-match hit


 The trading symbol for Amazon is AMZN.
Summarize the document


Llama.generate: prefix-match hit


 The document appears to be a financial report for Amazon, which includes consolidated financial statements such as statements of cash flows, operations, comprehensive income, balance sheets, and stockholders' equity. Additionally, there are notes to the financial statements that provide further information about the company's financial performance. The document also includes references to laws and regulations, and uses specific terms such as "asset" and "property" to describe various types of assets and properties owned by the company.
which place amazon was incorporated


Llama.generate: prefix-match hit


 Based on the given context information, Amazon was incorporated in Delaware.
Give the last quater revenue for amazon


Llama.generate: prefix-match hit


 Based on the provided context information, the last quarter revenue for Amazon is not explicitly stated. However, it can be calculated by taking the total revenue for the nine months ended September 30, 2021 and dividing it by 3 to get the revenue for one quarter.

The total revenue for the nine months ended September 30, 2021 is $5,345 million. Dividing this by 3 gives us a revenue of approximately $1,781.67 million per quarter.
